# 分布式进程

- 在Thread和Process中，应当优选Process，因为Process更稳定，而且，Process可以分布到多台机器上，而Thread最多只能分布到同一台机器的多个CPU上

- Python的multiprocess模块不但支持多进程，其中managers子模块还支持把多进程分布到多台机器上，一个服务进程可以作为调度者，将任务分布到其他多个进程中，依靠网络通信。由于managers模块封装很好，不必了解网络通信的细节，就可以很容易地编写分布式多进程程序。

## 实例

- 如果我们已经有一个通过Queue通信的多进程程序在同一台机器上运行，现在，由于处理任务的进程任务繁重，希望把发送任务的进程和处理任务的进程分布到两台机器上，怎么用分布式进程实现？
    - 原有的Queue可以继续使用，但是通过managers模块把Queue通过网络暴露出去，就可以让其他机器的进程访问Queue了

### 服务器进程负责启动Queue，把Queue注册到网络上，然后往Queue里面写入任务：

In [ ]:
# task_master.py
import random,time,queue
from multiprocessing.managers import BaseManager


# 发送任务的队列：
task_queue = queue.Queue()
# 接收结果的队列：
result_queue = queue.Queue()

# 从BaseManger继承的QueueManger:
class QueueManager(BaseManager):
    pass

# 把两个Queue都注册到网络上，callable参数关联了Queue对象：
QueueManager.register('get_task_queue', callable=lambda: task_queue)
QueueManager.register('get_result_queue', callable=lambda: result_queue)
# 绑定端口5000， 设置验证码‘abc':
manager = QueueManager(address=('', 5000), authkey=b'abc')
# 启动Queue:
manager.start()
# 获得通过网络访问的Queue对象：
task = manager.get_task_queue()
result = manager.get_result_queue()
# 放几个任务进去
for i in range(10):
    n = random.randint(0, 10000)
    print('Put task %d...' % n)
    task.put(n)

# 从result队列读取结果：
print('Try get results...')
for i in range(10):
    r = result.get(timeout=10)
    print('Result: {}'.format(r))

# 关闭
manager.shutdown()
print('master exit.')

- 当我们在一台机器上写多进程程序时，创建的Queue可以直接拿来用，但是，在分布式多进程环境下，添加任务到Queue不可以直接对原始的task_queue进行操作，那样就绕过了QueueManger的封装，必须通过manager.get_task_queue()获得的Queue接口添加

## 然后在另一台机器上启动任务进程（本机上启动也可以）：

In [ ]:
# task_worker.py
import time, sys, queue
from multiprocessing.managers import BaseManager


# 创建类似的QueueManager:
class QueueManager(BaseManager):
    pass

# 由于这个QueueManager只从网络上获取Queue，所以注册时只提供名字：
QueueManager.register('get_task_queue')
QueueManager.register('get_result_queue')

# 连接到服务器，也就是运行task_master.py的机器：
server_add = '127.0.0.1'
print('Connect to server {}...'.format(server_add))
# 端口和验证码注意保持与task_master.py设置的完全一致
m = QueueManager(address=(server_add, 5000), authkey=b'abc')
# 从网络连接：
m.connect()
# 获取Queue的对象：
task = m.get_task_queue()
result = m.get_result_queue()
# 从task队列取任务，并把结果写入result队列
for i in range(10):
    try:
        n = task.get(timeout=1)
        print('run task {d}*{d}...'.format(n, n))
        r = '{d} * {d} = {d}'.format(n, n, n*n)
        time.sleep(1)
        result.put(r)
    except Queue.Empty:
        print('task queue is empty.')
# 处理结束
print('worker exit.')

- 任务进程要通过网络连接到服务进程，所以要指定服务进程的IP

![](3.png)
![](4.png)

## 总结

- 以上这个简单的Master/Worker模型其实就是一个简单但真正的分布式计算，把代码稍加改造，启动多个worker，就可以把任务分布到几台甚至几十台机器上，比如把计算n*n的代码换成发送邮件，就实现了邮件队列的异步发送。

- Queue对象存储在task_master的进程中，task_worker中根本没有创建Queue的代码。

- 而Queue之所以能够通过网络访问，就是通过QueueManager实现的，由于QueueManager管理的不止一个Queue，所以，要给每个Queue的网络调用接口起个名字，比如get_task_queue

- authkey的用处：保证两台机器正常通信，不被其他机器恶意干扰。如果task_worker.py的authkey和task_master.py的authkey不一致，肯定连接不上。

- Python的分布式进程接口简单，封装良好，适合需要把繁重任务分布到多台机器的环境下。
- **注意Queue的作用是用来传递任务和接收结果，每个任务的描述数据量要尽量小，比如发送一个处理日志文件的任务，就不要发送几百M的日志文件本身，而是发送日志文件存放的完整路径，由worker进程再去共享的磁盘上读取文件。**